# CFG: Using rules to check grammar

The previous notebook demonstrated that a CFG can be used to generate sentences. CFG can also be used to check if an input sentence is grammatical. There are several algorithms such as CYK that can do this check. In this notebook we go through a few rounds of applying CFG grammar manually until we arrive at S.

Using the same rules as before: 

In [1]:
rules = {'S' : [['NP', 'VP']],
         'NP': [['DT', 'NN'], ['DT', 'JJ', 'NN']],
         'VP': [['VB'], ['VB', 'PP']],
         'PP': [['P', 'DT', 'NN']],
         'JJ' : ['happy', 'sad', 'silly'],
         'DT': ['a', 'the'],
         'NN': ['cat', 'dog', 'clown'],
         'VB': ['plays', 'runs'],
         'P' : ['with', 'near', 'beside']
        }

## Create a reversed dictionary so that items on rhs can be clasified quickly

In [2]:
# create a dictionary item:class

class_dict = {}
for rule in rules.keys():
    if isinstance(rules[rule][0], str): # terminals
        for token in rules[rule]:
            class_dict[token] = rule
    else:  # non-terminals
        for lst in rules[rule]:
            pattern = ' '.join(lst)
            class_dict[pattern] = rule
class_dict

{'NP VP': 'S',
 'DT NN': 'NP',
 'DT JJ NN': 'NP',
 'VB': 'VP',
 'VB PP': 'VP',
 'P DT NN': 'PP',
 'happy': 'JJ',
 'sad': 'JJ',
 'silly': 'JJ',
 'a': 'DT',
 'the': 'DT',
 'cat': 'NN',
 'dog': 'NN',
 'clown': 'NN',
 'plays': 'VB',
 'runs': 'VB',
 'with': 'P',
 'near': 'P',
 'beside': 'P'}

In [3]:
# round 1: for each input token, assign one or more POS
# in this example, there will be only one but in another set,
#   a word like 'senses' could be a noun or verb
    
sentence = 'the happy cat plays'
tokens = sentence.split()
patterns = [[class_dict[t]] for t in tokens]
patterns

[['DT'], ['JJ'], ['NN'], ['VB']]

In [4]:
# round 2: for each pattern, find any lhs rule that could have generated it
# now the tokens are taken 2 at a time
# in this example we only have one pos so the code is straightforward
indices = [[i, i+1] for i in range(0, len(tokens)-1)]
word_strings = [tokens[i]+' '+tokens[j] for i,j in indices]
search_patterns = [[patterns[i][0]+' '+patterns[j][0]] for i,j in indices]
print('words and patterns:', word_strings, search_patterns)

words and patterns: ['the happy', 'happy cat', 'cat plays'] [['DT JJ'], ['JJ NN'], ['NN VB']]


In [5]:
# search patterns for dictionary matches and replace when match is found, else X
new_patterns = []
for i, patt in enumerate(search_patterns):
    new_patt = []  # assume no match
    for p in patt:
        if p in class_dict:
            new_patt.append(class_dict[p])
    new_patterns.append(new_patt)
new_patterns

[[], [], []]

In [6]:
# round 3: none of the lenth-2 patterns match, try length 3
# since none of the patterns in round 2 matched we are just looking back at pos, only one choice
indices = [[i, i+2] for i in range(0, len(tokens)-2)]
word_strings = [tokens[i]+' '+tokens[i+1]+' '+tokens[j] for i,j in indices]
search_patterns = [[patterns[i][0]+' '+patterns[i+1][0]+' '+patterns[j][0]] for i,j in indices]
print('words and patterns:', word_strings, search_patterns)

words and patterns: ['the happy cat', 'happy cat plays'] [['DT JJ NN'], ['JJ NN VB']]


In [7]:
# search for maches
new_patterns = []
for i, patt in enumerate(search_patterns):
    new_patt = []  # assume no match
    for p in patt:
        if p in class_dict:
            new_patt.append(class_dict[p])
    new_patterns.append(new_patt)
new_patterns

[['NP'], []]

In [8]:
# round 4: 'the happy cat' can be replace by 'NP'
# the remainder of the sentence is 'plays' = 'VB'
search_patterns = patterns[3:]
for i, patt in enumerate(search_patterns):
    new_patt = []  # assume no match
    for p in patt:
        if p in class_dict:
            new_patt.append(class_dict[p])
    new_patterns.append(new_patt)
new_patterns

[['NP'], [], ['VP']]

### from here we can extract the pattern 'NP VP' which matches S
The sentence is established to be grammatical